In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:0"

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 32, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0.2):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(2, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
        
        self.lineartest0 = nn.Linear(2, emsize)
        self.lineartest1 = nn.Linear(2, emsize)
        self.lineartest2 = nn.Linear(2, emsize)
        self.lineartest3 = nn.Linear(2 * emsize, emsize)
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 2]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def test_encode(self, src): #src must be [batchsize * nsamples, npoints = 5, 2]
        point_1 = self.lineartest1(src[:, 3, :] - 0.5) #want mean 0!!!!
        point_2 = self.lineartest1(src[:, 4, :] - 0.5)
        remaining = self.lineartest0(src[:, :3, :]) #[batchsize * nsamples, 3, emsize]
        point_1_message = self.lineartest1(src[:, 3, :] - 0.5).squeeze(1)
        point_2_message = self.lineartest2(src[:, 4, :] - 0.5).squeeze(1)
        point_1 = F.relu(torch.cat([point_1, point_2_message], dim = 1))
        point_2 = F.relu(torch.cat([point_2, point_1_message], dim = 1))
        point_1 = self.lineartest3(point_1)
        point_2 = self.lineartest3(point_2)
        src = torch.cat([torch.zeros_like(remaining.transpose(0, 1)), point_1.unsqueeze(0), point_2.unsqueeze(0)])
        return src #[npoints = 5, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 3], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 3, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 3, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 3, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 3, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 3, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, dtype = torch.bool, device = device).triu(1) #[npoints - 3, npoints - 3], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 3, npoints - 3]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 3, npoints - 3]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment():
    def reset(self, npoints, batchsize, nsamples):
        if npoints <= 3:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = batchsize * nsamples #so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = torch.rand([batchsize, npoints - 3, 2], device = device).unsqueeze(1).expand(-1, nsamples, -1, -1).reshape(self.batchsize, npoints - 3, 2)
        self.corner_points = torch.tensor([[0, 0], [2, 0], [0, 2]], dtype = torch.float, device = device)
        self.points = torch.cat([self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1), self.points], dim = -2) #[batchsize * nsamples, npoints, 2]
        self.points_mask = torch.cat([torch.ones([self.batchsize, 3], dtype = torch.bool, device = device), torch.zeros([self.batchsize, npoints - 3], dtype = torch.bool, device = device)], dim = 1)
        self.points_sequence = torch.empty([self.batchsize, 0], dtype = torch.long, device = device)
        
        """use a trick, for the purpose of an 'external' triangle that is always left untouched, which means we don't have to deal with boundary edges as being different. external triangle is [0, 1, 2] traversed clockwise..."""
        self.partial_delaunay_triangles = torch.tensor([[0, 2, 1], [0, 1, 2]], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1, -1).contiguous() #[batchsize, ntriangles, 3] contains index of points, always anticlockwise
        self.partial_delaunay_edges = torch.tensor([5, 4, 3, 2, 1, 0], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1).contiguous() #[batchsize, ntriangles * 3] contains location of corresponding edge (edges go in order 01, 12, 20). Edges will always flip since triangles are stored anticlockwise.
        
        self.ntriangles = 2 #can store as scalar, since will always be the same
        self.cost = torch.zeros([self.batchsize], device = device)
        
        self.logprob = torch.zeros([self.batchsize], device = device, requires_grad = True)
    
    def update(self, point_index): #point_index is [batchsize]
        if point_index.size(0) != self.batchsize:
            print("Error: point_index.size() doesn't match expected size, should be [batchsize]")
            return
        if self.points_mask.gather(1, point_index.unsqueeze(1)).sum():
            print("Error: some points already added")
            return
        triangles_coordinates = self.points.gather(1, self.partial_delaunay_triangles.view(self.batchsize, self.ntriangles * 3).unsqueeze(2).expand(-1, -1, 2)).view(self.batchsize, self.ntriangles, 3, 2) #[batchsize, ntriangles, 3, 2]
        newpoint = self.points.gather(1, point_index.unsqueeze(1).unsqueeze(2).expand(self.batchsize, 1, 2)).squeeze(1) #[batchsize, 2]
        
        incircle_matrix = torch.cat([triangles_coordinates, newpoint.unsqueeze(1).unsqueeze(2).expand(-1, self.ntriangles, 1, -1)], dim = -2) #[batchsize, ntriangles, 4, 2]
        incircle_matrix = torch.cat([incircle_matrix, (incircle_matrix * incircle_matrix).sum(-1, keepdim = True), torch.ones([self.batchsize, self.ntriangles, 4, 1], device = device)], dim = -1) #[batchsize, ntriangles, 4, 4]
        incircle_test = incircle_matrix.det() > 0 #[batchsize, ntriangles], is True if inside incircle
        removed_edge_mask = incircle_test.unsqueeze(2).expand(-1, -1, 3).reshape(-1) #[batchsize * ntriangles * 3]
        
        edges = (self.partial_delaunay_edges + self.ntriangles * 3 * torch.arange(self.batchsize, device = device).unsqueeze(1)).view(-1) #[batchsize * ntriangles * 3]
        neighbouring_edge = edges.masked_select(removed_edge_mask)
        neighbouring_edge_mask = torch.zeros([self.batchsize * self.ntriangles * 3], device = device, dtype = torch.bool)
        neighbouring_edge_mask[neighbouring_edge] = True
        neighbouring_edge_mask = (neighbouring_edge_mask * removed_edge_mask.logical_not()) #[batchsize * ntriangles * 3]
        
        n_new_triangles = neighbouring_edge_mask.view(self.batchsize, -1).sum(-1) #[batchsize]
        
        new_point = point_index.unsqueeze(1).expand(-1, self.ntriangles * 3).masked_select(neighbouring_edge_mask.view(self.batchsize, -1))
        
        second_point_mask = neighbouring_edge_mask.view(self.batchsize, -1, 3) #[batchsize, ntriangles 3]
        (first_point_indices0, first_point_indices1, first_point_indices2) = second_point_mask.nonzero(as_tuple = True)
        first_point_indices2 = (first_point_indices2 != 2) * (first_point_indices2 + 1)
        
        first_point = self.partial_delaunay_triangles[first_point_indices0, first_point_indices1, first_point_indices2] #[?]
        second_point = self.partial_delaunay_triangles.masked_select(second_point_mask) #[?]
        
        new_triangles_mask = torch.cat([incircle_test, torch.ones([self.batchsize, 2], dtype = torch.bool, device = device)], dim = 1) #[batchsize, ntriangles + 2]
        
        new_neighbouring_edges = 3 * new_triangles_mask.nonzero(as_tuple = True)[1] #[?], 3* since is the 01 edge of new triangles (see later)
        self.partial_delaunay_edges.masked_scatter_(neighbouring_edge_mask.view(self.batchsize, -1), new_neighbouring_edges) #still [batchsize, ntriangles * 3] for now
        
        self.partial_delaunay_triangles = torch.cat([self.partial_delaunay_triangles, torch.empty([self.batchsize, 2, 3], dtype = torch.long, device = device)], dim = 1)
        self.partial_delaunay_edges = torch.cat([self.partial_delaunay_edges, torch.empty([self.batchsize, 6], dtype = torch.long, device = device)], dim = 1)
        new_triangles = torch.stack([first_point, second_point, new_point], dim = 1) #[?, 3], edge here is flipped compared to edge in neighbouring triangle (so first_point is the second point in neighbouring edge)
        self.partial_delaunay_triangles.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3), new_triangles) #[batchsize, ntriangles + 2, 3]
        
        new_edge01 = neighbouring_edge_mask.view(self.batchsize, -1).nonzero(as_tuple = True)[1] #[?]
        
        """we are currently storing which triangles have to be inserted, via the edges along the perimeter of the delaunay cavity, we need to compute which edge is to the 'left'/'right' of each edge"""
        """don't have the memory to do a batchsize * n * n boolean search, don't have the speed to do a batchsize^2 search (as would be the case for sparse matrix or similar)"""
        """best alternative: rotate the edge around right point, repeat until hit edge in mask (will never go to an edge of a removed triangle before we hit edge in mask) should basically be order 1!!!!!"""
        
        neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?]
        next_neighbouring_edge_index = torch.empty_like(neighbouring_edge_index) #[?]
        
        rotating_flipped_neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?], initialise
        todo_mask = torch.ones_like(next_neighbouring_edge_index, dtype = torch.bool) #[?]
        while todo_mask.sum():
            rotating_neighbouring_edge_index = rotating_flipped_neighbouring_edge_index + 1 - 3 * (rotating_flipped_neighbouring_edge_index % 3 == 2) #[todo_mask.sum()], gets smaller until nothing left EFFICIENCY (this may be seriously stupid, as it requires making a bunch of copies when I could be doing stuff inplace)
            
            update_mask = neighbouring_edge_mask[rotating_neighbouring_edge_index] #[todo_mask.sum()]
            update_mask_unravel = torch.zeros_like(todo_mask).masked_scatter(todo_mask, update_mask) #[?]
            
            next_neighbouring_edge_index.masked_scatter_(update_mask_unravel, rotating_neighbouring_edge_index.masked_select(update_mask)) #[?]
            
            todo_mask.masked_fill_(update_mask_unravel, False) #[?]
            rotating_flipped_neighbouring_edge_index = edges[rotating_neighbouring_edge_index.masked_select(update_mask.logical_not())] #[todo_mask.sum()]
        triangle_index = new_triangles_mask.view(-1).nonzero(as_tuple = True)[0] #[?], index goes up to batchsize * (ntriangles + 2), this is needed for when we invert the permutation by scattering (won't scatter same number of triangles per batch)
        
        next_triangle_index = torch.empty_like(edges).masked_scatter_(neighbouring_edge_mask, triangle_index)[next_neighbouring_edge_index] #[?], index goes up to batchsize * (ntriangles + 2)
        next_edge = 3 * next_triangle_index + 1 #[?]
        
        invert_permutation = torch.empty_like(new_triangles_mask.view(-1), dtype=torch.long) #[batchsize * (ntriangles + 2)]
        invert_permutation[next_triangle_index] = triangle_index #[batchsize * (ntriangles + 2)]
        previous_triangle_index = invert_permutation.masked_select(new_triangles_mask.view(-1)) #[?]
        previous_edge = 3 * previous_triangle_index + 2 #[?]
        
        """in the above we rotated around 'first_point' in our new triangles"""
        new_edge20 = next_edge % ((self.ntriangles + 2) * 3) #[?]
        new_edge12 = previous_edge % ((self.ntriangles + 2) * 3) #[?]
        
        new_edges = torch.stack([new_edge01, new_edge12, new_edge20], dim = 1) #[?, 3]
        self.partial_delaunay_edges.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3).reshape(self.batchsize, -1), new_edges) #[batchsize, (ntriangles + 2) * 3]
        
        self.ntriangles += 2
        """currently only count the extra triangles you replace (not the on you have to remove because you're located there, and not the ones you make because you have to create two more"""
        self.cost += (n_new_triangles - 3)
        self.points_mask.scatter_(1, point_index.unsqueeze(1).expand(-1, self.npoints), True)
        self.points_sequence = torch.cat([self.points_sequence, point_index.unsqueeze(1)], dim = 1)
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 300000, npoints = 13, batchsize = 2000, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(3, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 0.5
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        loss = ( 
                - logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
                * positive_reinforcement_binary 
                + logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 6) 
                * negative_reinforcement_binary 
        ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
  

In [ ]:
train(npoints = 13, batchsize = 3500, nsamples = 8)

12.925999641418457 -14.194711685180664 -15.060077667236328 -14.950976371765137 -14.740638732910156 -14.740640640258789
12.410285949707031 -13.577527046203613 -14.955879211425781 -15.543980598449707 -14.803855895996094 -14.803854942321777
12.06257152557373 -12.666804313659668 -14.669846534729004 -14.629585266113281 -15.235827445983887 -15.235827445983887
11.853713989257812 -11.435879707336426 -14.018937110900879 -14.507492065429688 -12.062093734741211 -12.062094688415527
11.872285842895508 -11.661133766174316 -14.055187225341797 -13.67674446105957 -14.317493438720703 -14.317493438720703
11.807714462280273 -11.608274459838867 -13.975211143493652 -14.475229263305664 -12.302146911621094 -12.30214786529541
11.888285636901855 -11.598270416259766 -13.934308052062988 -16.17840003967285 -14.969392776489258 -14.969392776489258
11.766857147216797 -10.938057899475098 -13.521032333374023 -12.330545425415039 -13.800459861755371 -13.800461769104004
11.741142272949219 -10.784160614013672 -13.580239295

11.127142906188965 -9.632224082946777 -13.10713005065918 -11.206716537475586 -11.767187118530273 -11.76718807220459
11.156000137329102 -9.73465347290039 -13.175834655761719 -12.228878021240234 -13.368884086608887 -13.368884086608887
11.062000274658203 -9.689154624938965 -13.148712158203125 -11.670754432678223 -11.726095199584961 -11.726094245910645
11.058570861816406 -9.677752494812012 -13.145101547241211 -14.981691360473633 -17.21352767944336 -17.21352767944336
11.066571235656738 -9.810060501098633 -13.183340072631836 -10.92980670928955 -9.455673217773438 -9.455674171447754
11.086856842041016 -9.943242073059082 -13.306498527526855 -14.770964622497559 -11.864529609680176 -11.864530563354492
11.050285339355469 -10.140981674194336 -13.428780555725098 -13.765426635742188 -14.899124145507812 -14.899126052856445
11.145713806152344 -10.354190826416016 -13.568800926208496 -11.833822250366211 -11.071233749389648 -11.071235656738281
11.08657169342041 -10.327790260314941 -13.564384460449219 -15.

10.929428100585938 -9.908764839172363 -13.534871101379395 -11.781270980834961 -15.483243942260742 -15.483243942260742
11.003999710083008 -9.77563190460205 -13.445290565490723 -12.682478904724121 -14.777875900268555 -14.777875900268555
10.989142417907715 -9.637356758117676 -13.382697105407715 -12.876115798950195 -17.87063217163086 -17.87063217163086
10.932856559753418 -9.6057767868042 -13.358627319335938 -14.184466361999512 -13.078551292419434 -13.07855224609375
10.935999870300293 -9.698229789733887 -13.467489242553711 -12.068893432617188 -12.099381446838379 -12.099379539489746
10.934000015258789 -9.851682662963867 -13.541657447814941 -14.889900207519531 -12.288850784301758 -12.288850784301758
10.832857131958008 -10.004798889160156 -13.637809753417969 -14.047131538391113 -13.081140518188477 -13.08113956451416
10.89371395111084 -9.989787101745605 -13.577593803405762 -10.581206321716309 -16.21826934814453 -16.21826934814453
10.866571426391602 -9.915955543518066 -13.506611824035645 -13.342

10.751999855041504 -9.574408531188965 -13.35929012298584 -13.58504867553711 -14.477253913879395 -14.477254867553711
10.784285545349121 -9.689485549926758 -13.43761157989502 -13.744230270385742 -13.96688175201416 -13.966882705688477
10.757143020629883 -9.810033798217773 -13.527657508850098 -13.516324996948242 -14.684318542480469 -14.684318542480469
10.800285339355469 -9.881061553955078 -13.599183082580566 -16.254173278808594 -11.765162467956543 -11.765161514282227
10.720000267028809 -9.671142578125 -13.44314193725586 -11.496170997619629 -15.483321189880371 -15.483322143554688
10.676570892333984 -9.56016731262207 -13.347343444824219 -11.584671020507812 -14.004794120788574 -14.004794120788574
10.765714645385742 -9.305252075195312 -13.141234397888184 -12.866472244262695 -11.210309982299805 -11.210309982299805
10.809714317321777 -9.175310134887695 -13.057493209838867 -12.722835540771484 -12.365025520324707 -12.365025520324707
10.769143104553223 -9.354229927062988 -13.186790466308594 -16.697

10.5977144241333 -9.455720901489258 -13.30229663848877 -15.647459030151367 -11.735523223876953 -11.735523223876953
10.618857383728027 -9.497138023376465 -13.327310562133789 -17.577672958374023 -12.749860763549805 -12.749860763549805
10.54171371459961 -9.51664924621582 -13.376203536987305 -12.790555953979492 -13.592020988464355 -13.592020988464355
10.665714263916016 -9.577936172485352 -13.45361328125 -15.22966194152832 -12.429823875427246 -12.429824829101562
10.562285423278809 -9.752975463867188 -13.540234565734863 -12.761871337890625 -16.83723258972168 -16.83723258972168
10.610285758972168 -9.78954029083252 -13.556318283081055 -13.760746955871582 -11.947993278503418 -11.947993278503418
10.589142799377441 -9.80325698852539 -13.571930885314941 -9.957807540893555 -14.80076789855957 -14.800768852233887
10.595714569091797 -9.855422973632812 -13.59967041015625 -14.664125442504883 -15.194008827209473 -15.194009780883789
10.623143196105957 -9.766051292419434 -13.551669120788574 -11.74546432495

...

9.937999725341797 -9.410030364990234 -13.394336700439453 -13.509246826171875 -11.009515762329102 -11.009514808654785
10.013999938964844 -9.43613338470459 -13.390542984008789 -11.10599136352539 -15.29262924194336 -15.29262924194336
9.991714477539062 -9.451593399047852 -13.379386901855469 -14.941431045532227 -13.889451026916504 -13.889451026916504
9.955714225769043 -9.49692153930664 -13.390291213989258 -13.588696479797363 -11.703829765319824 -11.703829765319824
9.947428703308105 -9.49072551727295 -13.423808097839355 -12.613664627075195 -13.489397048950195 -13.489398956298828
9.98799991607666 -9.487618446350098 -13.39376449584961 -13.529302597045898 -15.497424125671387 -15.49742317199707
9.953143119812012 -9.48443603515625 -13.381479263305664 -14.572216987609863 -12.745633125305176 -12.74563217163086
9.968571662902832 -9.434966087341309 -13.3612699508667 -15.71401596069336 -15.682451248168945 -15.682450294494629
9.926856994628906 -9.384526252746582 -13.329437255859375 -12.210633277893066 

9.960857391357422 -9.557100296020508 -13.46657943725586 -12.393348693847656 -12.784135818481445 -12.784135818481445
9.962285995483398 -9.518439292907715 -13.485816955566406 -13.139629364013672 -15.824884414672852 -15.824886322021484
9.935999870300293 -9.440136909484863 -13.402203559875488 -16.662822723388672 -12.98219108581543 -12.982189178466797
9.96142864227295 -9.434041023254395 -13.411572456359863 -14.371929168701172 -13.962350845336914 -13.962352752685547
9.977428436279297 -9.396124839782715 -13.358772277832031 -12.729205131530762 -11.806975364685059 -11.806973457336426
9.951142311096191 -9.396459579467773 -13.3313570022583 -14.587553977966309 -14.646347045898438 -14.646347999572754
9.92199993133545 -9.433445930480957 -13.390876770019531 -13.483150482177734 -13.237662315368652 -13.237662315368652
9.942856788635254 -9.639815330505371 -13.50675106048584 -15.000421524047852 -12.275611877441406 -12.275611877441406
9.925999641418457 -9.693861961364746 -13.566701889038086 -12.8487863540

10.008285522460938 -9.648869514465332 -13.564913749694824 -14.832295417785645 -12.963850021362305 -12.963850975036621
9.934285163879395 -9.579278945922852 -13.497498512268066 -16.388736724853516 -14.191068649291992 -14.191068649291992
9.883999824523926 -9.43337631225586 -13.435988426208496 -12.237715721130371 -12.261856079101562 -12.26185417175293
9.937142372131348 -9.3751802444458 -13.363101959228516 -12.641014099121094 -12.609557151794434 -12.60955810546875
9.918571472167969 -9.31884479522705 -13.336029052734375 -12.623044967651367 -14.37631607055664 -14.376315116882324
9.908285140991211 -9.365424156188965 -13.346464157104492 -16.23062515258789 -12.020124435424805 -12.020125389099121
9.892285346984863 -9.3921537399292 -13.377522468566895 -15.071022987365723 -12.829679489135742 -12.829679489135742
9.89371395111084 -9.554402351379395 -13.469507217407227 -17.191038131713867 -12.919081687927246 -12.919082641601562
9.987142562866211 -9.655832290649414 -13.549734115600586 -15.4121589660644

9.886285781860352 -9.693523406982422 -13.568402290344238 -12.801107406616211 -14.120477676391602 -14.120477676391602
9.894000053405762 -9.654852867126465 -13.5567626953125 -11.595808029174805 -14.278129577636719 -14.278128623962402
9.921713829040527 -9.552329063415527 -13.481945991516113 -13.168889999389648 -13.023935317993164 -13.023935317993164
9.894571304321289 -9.497966766357422 -13.453507423400879 -11.74571704864502 -12.923639297485352 -12.923639297485352
9.898571014404297 -9.477649688720703 -13.421876907348633 -14.98327922821045 -9.866056442260742 -9.866057395935059
9.937142372131348 -9.489761352539062 -13.46529769897461 -17.263408660888672 -11.49423599243164 -11.494235038757324
9.895142555236816 -9.521366119384766 -13.481348037719727 -12.173545837402344 -11.33131217956543 -11.33131217956543
9.8628568649292 -9.51999568939209 -13.483837127685547 -12.824042320251465 -14.136728286743164 -14.136727333068848
9.89028549194336 -9.473806381225586 -13.438779830932617 -13.414432525634766 -

9.958856582641602 -9.475400924682617 -13.41655445098877 -13.008673667907715 -14.471336364746094 -14.471336364746094
9.871999740600586 -9.505928039550781 -13.418712615966797 -11.94034481048584 -17.586952209472656 -17.586952209472656
9.857142448425293 -9.461870193481445 -13.412823677062988 -12.911495208740234 -14.81212043762207 -14.81212043762207
9.918856620788574 -9.339814186096191 -13.31436824798584 -14.450698852539062 -14.292104721069336 -14.292104721069336
9.903428077697754 -9.30515193939209 -13.317792892456055 -13.397340774536133 -13.071662902832031 -13.071663856506348
9.884857177734375 -9.341802597045898 -13.30886459350586 -11.181295394897461 -13.324539184570312 -13.324541091918945
9.85085678100586 -9.388111114501953 -13.375317573547363 -13.590786933898926 -19.350778579711914 -19.350778579711914
9.910571098327637 -9.590420722961426 -13.463693618774414 -10.8302640914917 -14.747074127197266 -14.747074127197266
9.858285903930664 -9.655845642089844 -13.529338836669922 -10.8095989227294